In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# googleplaystore.csv: https://drive.google.com/file/d/1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ/view?usp=drive_link
# googleplaystore_user_reviews.csv: https://drive.google.com/file/d/1XX4S3LUe_bVB94g5H2tv5rhEANeSX_sO/view?usp=drive_link
downloaded = drive.CreateFile({'id':"1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ"})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context -> variable que vamos a utilizar
sc = spark.sparkContext

Juan tiene un dispositivo un poco viejo con Android 3.9 y ya no lo puede actualizar. Quiere descargar apps del género de “Arte y Diseño” pero no sabe cuáles son válidas para su versión de Android. Ayúdalo diciéndole cuántas apps puede descargarse y mencionale 3 de ellas con su respectivo Rating. (⭐)

In [ ]:
def compatible_android_ver(version):

  version_numbers = version.split('.')
  android_ver_float = float(f"{version_numbers[0]}.{version_numbers[1]}")
  # si la version es 3.9.n, solo es compatible si n es 0
  if android_ver_float == 3.9 and len(version_numbers) > 2:
    if version_numbers[2] > 0:
      return False
    else:
      return True
  return android_ver_float < 3.9

In [ ]:
import math
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
rdd = df.rdd

filtered_rdd = rdd.filter(lambda x: ('Art & Design' in x.Genres))

Decido descartar las apps que no especifican la version de android (no sirven para comparar)


In [ ]:
filtered_rdd = filtered_rdd.filter(lambda x: x['Android Ver'] != 'Varies with device')

Me quedo solo con el numero de version (elimino el ' and up')


In [ ]:
clean_rdd = filtered_rdd.map(lambda x: (x['App'], x['Rating'], x['Genres'], x['Android Ver'].replace(' and up', '').strip()))

Para mostrar elijo solo las que tienen Rating distinto a NaN


In [ ]:
compatible_apps_rdd = clean_rdd.filter(lambda x: compatible_android_ver(x[3]))
print(f"\nJuan, podes descargarte: {compatible_apps_rdd.count()} apps del genero de 'Arte y Diseño' que son compatibles con tu version de Android\n")
print("Por ejemplo, podes descargarte las siguientes 3 apps:")
sample = compatible_apps_rdd.filter(lambda x: not math.isnan(x[1])).takeSample(False, 3)
for app in sample:
  print(f"App: {app[0]}, Rating: {app[1]}")


Juan, podes descargarte: 11 apps del genero de 'Arte y Diseño' que son compatibles con tu version de Android

Por ejemplo, podes descargarte las siguientes 3 apps:
App: Art Drawing Ideas, Rating: 4.1
App: Easy Realistic Drawing Tutorial, Rating: 4.1
App: Harley Quinn wallpapers HD, Rating: 4.8
